In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

In [3]:
event_data = pd.read_csv('/Users/cram/Downloads/event_data_train.csv')

In [4]:
submissions_data = pd.read_csv('/Users/cram/Downloads/submissions_data_train.csv')

In [5]:
event_data['data']=pd.to_datetime(event_data.timestamp, unit = 's')

In [6]:
event_data['day'] = event_data.data.dt.date

In [7]:
users_events_data = event_data.pivot_table(index = 'user_id', 
                       columns = 'action', 
                       values = 'step_id', 
                       aggfunc = 'count',
                       fill_value = 0).reset_index().head()

In [8]:
submissions_data['data']=pd.to_datetime(submissions_data.timestamp, unit = 's')
submissions_data['day'] = event_data.data.dt.date

In [9]:
users_scores = submissions_data.pivot_table(index = 'user_id', 
                                           columns = 'submission_status', 
                                           values = 'step_id', 
                                           aggfunc = 'count',
                                           fill_value = 0).reset_index()

In [10]:
gape_data = event_data[['user_id','day','timestamp']].drop_duplicates(subset=['user_id','day']) \
.groupby('user_id')['timestamp'].apply(list) \
.apply(np.diff).values

In [11]:
gape_data = pd.Series(np.concatenate(gape_data, axis = 0))

In [12]:
gape_data =  gape_data / (24*60*60)

In [13]:
users_data = event_data.groupby('user_id', as_index = False) \
.agg({'timestamp':'max'}).rename(columns = {'timestamp':'last_timestamp'})

In [14]:
now = 1526772811
drop_out = 24*60*60*30

In [15]:
users_data['is_gone_users'] = (now - users_data.last_timestamp) > drop_out

In [16]:
users_data = users_data.merge(users_scores, on='user_id', how = 'outer')
users_data = users_data.fillna(0)

In [17]:
users_data.head()

,user_id,last_timestamp,is_gone_users,correct,wrong
0,1,1472827464,True,0.0,0.0
1,2,1519226966,True,2.0,0.0
2,3,1444581588,True,29.0,23.0
3,5,1499859939,True,2.0,2.0
4,7,1521634660,True,0.0,0.0


In [18]:
users_data = users_data.merge(users_events_data, how = 'outer')

In [19]:
users_days = event_data.groupby('user_id').day.nunique().to_frame().reset_index()

In [20]:
users_data = users_data.merge(users_days, how = 'outer')

In [21]:
users_data.head()

,user_id,last_timestamp,is_gone_users,correct,wrong,discovered,passed,started_attempt,viewed,day
0,1,1472827464,True,0.0,0.0,1.0,0.0,0.0,1.0,1
1,2,1519226966,True,2.0,0.0,9.0,9.0,2.0,10.0,2
2,3,1444581588,True,29.0,23.0,91.0,87.0,30.0,192.0,7
3,5,1499859939,True,2.0,2.0,11.0,11.0,4.0,12.0,2
4,7,1521634660,True,0.0,0.0,1.0,1.0,0.0,1.0,1


In [22]:
users_data.user_id.nunique()

19234

In [24]:
users_data['passed_course']= users_data.passed > 170

In [25]:
users_data.groupby('passed_course').count()

,user_id,last_timestamp,is_gone_users,correct,wrong,discovered,passed,started_attempt,viewed,day
passed_course,,,,,,,,,,
False,19234,19234,19234,19234,19234,5,5,5,5,19234
